# Workshop_08
## Immagine di esempio:

Dopo aver scelto un immagine su internet, è stato utilizzato "inkScape" per ricalcare la figura di riferimento nelle sue parti, in modo da creare dei file .svg da tradurre in file .lines.

<img src='./esempio.PNG'>

## Codice:
Vengono usate due funzioni:
* Una funzione primaria *ggpl_create_house()* che costruisce a partire da alcuni ".lines" files embeddati nel codice la riproduzione in 3d di una casa a partire dalla sua planimetria.

* Una funzione ausiliaria *get_lines_from_csv()* che si occupa di restituire la struct 2D delle linee presenti all'interno di ogni file ".lines"

In particolare sono stati utilizzati questi tipi di file .lines:
* muri esterni
* muri interni
* muri terrazzo
* porte
* finestre
* pavimenti bagni, cucina, esterni, moquette, legno (per definire alcune texture in base all'ambiente)

In [2]:
import csv
from pyplasm import *


"""ggpl_create_house() è una funzione che crea la struttura di una casa in 3d a partire
dalla planimetria della struttura."""
def ggpl_create_house():
    
    #AUX: return struct 2d cell complex from csv --------------------------------------

    def get_lines_from_csv(filename):
        lines = [] #[[x1,y1,x2,y2],[...]]

        file = open("./lines/"+ filename,'rt')
        try:
            reader = csv.reader(file)
            for row in reader:
                line = []
                for n in row:
                    line.append(float(n))
                lines.append(line)
        finally:
            file.close()

        polyline = []

        for line in lines:
            polyline.append(POLYLINE([[line[0],line[1]],[line[2],line[3]]]))

        return STRUCT(polyline)
    #END AUX ---------------------------------------------------------------------------

    #muri esterni -----------------------
    extWalls = get_lines_from_csv('muri esterni casa.lines')
    extWalls = PROD([OFFSET([4,4])(extWalls), Q(40)])

    #muri terrazzo --------------
    porWalls = get_lines_from_csv('muri esterni terrazzo.lines')
    porWalls = PROD([OFFSET([3,3])(porWalls), Q(10)]) #3d
    porWalls = DIFFERENCE([porWalls, extWalls])

    #muri interni -----------------------
    intWalls = get_lines_from_csv('muri interni casa.lines')
    intWalls = PROD([OFFSET([3,3])(intWalls), Q(40)]) #3d
    intWalls = DIFFERENCE([intWalls, extWalls])

    #buchi porte ------------------------
    doors = get_lines_from_csv('porte.lines')
    doors = SOLIDIFY(doors)
    doors = PROD([doors, Q(30)]) #3d

    #buchi finestre ---------------------
    windows = get_lines_from_csv('finestre.lines')
    windows = SOLIDIFY(windows)
    windows = T(3)(10)(PROD([windows, Q(20)])) #3d

    #pavimenti legno --------------------
    woodFloors = get_lines_from_csv('pavimenti legno.lines')
    woodFloors = TEXTURE("textures/parquet.jpg")(SOLIDIFY(woodFloors))

    #pavimenti moquette -----------------
    moqFloors = get_lines_from_csv('pavimenti moquette.lines')
    moqFloors = TEXTURE("textures/moquette.jpg")(SOLIDIFY(moqFloors))

    #pavimenti bagno --------------------
    wcFloors = get_lines_from_csv('pavimenti bagni.lines')
    wcFloors = TEXTURE("textures/wc.jpg")(SOLIDIFY(wcFloors))

    #pavimenti cucina -------------------
    kitFloors = get_lines_from_csv('pavimenti cucina.lines')
    kitFloors = TEXTURE(['textures/kitchen.jpg',True, False, 1, 1, PI/2., 5,5])(SOLIDIFY(kitFloors))

    #pavimenti esterni ------------------
    extFloors = get_lines_from_csv('pavimenti esterni.lines')
    extFloors = TEXTURE(['textures/outside.jpg',True, False, 1, 1, PI/2., 5,10])(SOLIDIFY(extFloors))

    #unisco pavimenti
    floors = STRUCT([woodFloors, wcFloors, kitFloors, extFloors, moqFloors])

    frame = STRUCT([extWalls, intWalls, porWalls])
    frame = DIFFERENCE([frame, windows, doors, floors]) #creo buchi per porte e finestre
    home = (STRUCT([frame, floors]))

    #scale
    x = 20/SIZE([1])(home)[0]
    y = 16/SIZE([2])(home)[0]
    z = 4/SIZE([3])(home)[0]
    
    home = S([1,2,3])([x,y,z])(STRUCT([frame, floors]))
    
    return home


In [ ]:
VIEW(ggpl_create_house())

<img src='img1.PNG'>
<img src='img2.PNG'>
<img src='img3.PNG'>